In [23]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from onnx.helper import make_graph, make_model, set_model_props
from onnx import helper
import numpy as np
import onnx

def get_init(model, tensor_name):
    for init in model.graph.initializer:
        if tensor_name == init.name:
            init_val = init.raw_data
            break
    return np.int64(int.from_bytes(init_val, byteorder='little'))
    
def add_attr(model, node):
    for ind,n in enumerate(model.graph.node):
        new_attr = []
        if node in n.name:            
            new_attr.append(helper.make_attribute("starts", get_init(model,n.input[1])))
            new_attr.append(helper.make_attribute("ends", get_init(model,n.input[2])))
            new_attr.append(helper.make_attribute("axes", get_init(model,n.input[3])))
            if len(n.input[1:]) >3:
                new_attr.append(helper.make_attribute("steps", get_init(model,n.input[4])))
            for na in new_attr:
                n.attribute.append(na)
    return model


def del_node_inputs(model, node):
    select_nodes = []
    remove_inputs = []
    for n in model.graph.node:
        if node in n.name:
            select_nodes.append(n)
            remove_inputs.append(n.input[1:])
            del n.input[1:]
    return model

   
def new_model(model):
    """
    Overwrites the main opset in an ONNX file.
    Does not change any node definition.
    :param model: ONNX model
    :param new_opset: new opset
    :return: ONNX model
    """    
    
    model_config = {}
    model_config["opset_imports"] = [onnx.helper.make_operatorsetid("",9)]
    graph = make_graph(
        model.graph.node, model.graph.name, model.graph.input,
        model.graph.output, model.graph.initializer)
#         sparse_initializer=model.graph.sparse_initializer)
    onnx_model = make_model(graph, functions=model.functions,**model_config)
    onnx_model.ir_version = model.ir_version
    onnx_model.producer_name = model.producer_name
    onnx_model.producer_version = model.producer_version
    onnx_model.domain = model.domain
    onnx_model.model_version = model.model_version
    onnx_model.doc_string = model.doc_string
    if len(model.metadata_props) > 0:  # pragma: no cover
        values = {p.key: p.value for p in model.metadata_props}
        set_model_props(onnx_model, values)

#     del onnx_model.opset_import[:]  # pylint: disable=E1101
#     for oimp in model.opset_import:
#         op_set = onnx_model.opset_import.add()  # pylint: disable=E1101
#         op_set.domain = oimp.domain
#         op_set.version = new_opset if oimp.domain == '' else oimp.version
    return ModelWrapper(onnx_model)


In [ ]:
model = ModelWrapper('tinyyolo-20210831.onnx')

In [28]:
model = ModelWrapper('tinyyolo-20210831.onnx')

model = add_attr(model, "Slice")
# model = add_domain(model,"Slice")
model = del_node_inputs(model, "Slice")
model = new_model(model.model)
model = model.transform(InferShapes())
model.save("tinyyolo-20210831_updated.onnx")


In [29]:
! python build.py

Building dataflow accelerator from tinyyolo-20210831_updated.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in build-tinyyolo-20210831_updated
Build log is at build-tinyyolo-20210831_updated/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 243, in step_tidy_up
    model = model.transform(FoldConstants())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/fold_constants.py", line 59, in apply
    oxe.execute_node(n, execution_context, graph)
  File "/workspace/qonnx/src/qonnx/core/onnx_exec.py", l